In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pycwt as wavelet
import matplotlib as mpl
import cdflib
from scipy import integrate
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad, quad_vec
from joblib import Parallel, delayed
from tqdm import tqdm

# Constants
R = 0.01369                   #Rigidity of the particle
B_0 = 235                     #mean Magnetic field
v = 0.01460 * 3e10  # m/s     #velocity of the particle
l_slab_m = 2 * 10**9          #slab bendover scale

Integrated PSD = 11432.079    #normalized PSD integrated with respect to normalized wavenumber

# Extract original data
tau = np.array(df['tau'])                              #\tau = \Omega * t, \Omega = relativistic gyro frequency, t is the time period of frequency
k_normalized = np.array(df['Normalized_Wavenumber'])   #k_normalized is l_slab_m * wavenumber(k)
g_slab = np.array(df['PSD_Normalized'])                #normalized Power spectrum of the magnetic field

# Reverse k_normalized for integration from smallest to largest
k_normalized_reversed = k_normalized[::-1]

# Define \widetilde{\sigma_z}(\tau) function
def sigma_z(tau):
    return 4 * np.pi * 11432.079 * (R / B_0)**2 * (2 - 2 * np.cos(tau) - 2 * tau * np.sin(tau) + tau**2)

# Define the integrand for the inner integral over \tau
def integrand_tau(tau, x, R, mu):
    sigma_z_val = sigma_z(tau)
    return np.exp(-(x**2 * sigma_z_val) / 2) * (
        np.cos((x * R * mu + 1) * tau) + np.cos((x * R * mu - 1) * tau)
    )

# Define the integrand for the outer integral over x
def integrand_x(x, R, mu, tau_segments):
    result = 0
    for start, end in tau_segments:
        inner_integral, _ = quad(integrand_tau, start, end, args=(x, R, mu))
        result += inner_integral
    g_slab_value = np.interp(x, k_normalized_reversed, g_slab[::-1])
    return g_slab_value * result

# Compute \tilde{D}_{\mu \mu }^{(2)}
def D_mu_mu(mu, R, B_0, k_segments, tau_segments):
    result = 0
    for start, end in k_segments:
        integral_x, _ = quad(lambda x: integrand_x(x, R, mu, tau_segments), start, end)
        result += integral_x
    return 2 * np.pi * ((1 - mu**2) / B_0**2) * (1 / R) * result

# Define the range of mu values
mu_values = np.linspace(0, 1, 10)

# Define the segments for tau and k_normalized
tau_segments = [(tau[::-1][i], tau[::-1][i+1]) for i in range(len(tau)-1)]
k_segments = [(k_normalized_reversed[i], k_normalized_reversed[i+1]) for i in range(len(k_normalized_reversed)-1)]

# Calculate D_mu_mu for each mu value using parallel computation
D_values = Parallel(n_jobs=-1)(delayed(D_mu_mu)(mu, R, B_0, k_segments, tau_segments) for mu in tqdm(mu_values, desc="Computing D_mu_mu"))

# Function to compute the integrand for kappa_parallel
def kappa_integrand(mu, D_values):
    D_mu = np.interp(mu, mu_values, D_values)
    if D_mu == 0 or np.isnan(D_mu):
        return 0  # Return 0 to avoid division by zero
    return (1 - mu**2)**2 / D_mu

# Calculate kappa_parallel
kappa_parallel, _ = quad_vec(kappa_integrand, 0, 1, args=(D_values,))

# Final kappa_parallel value
kappa_parallel_100KeV = (v * l_slab_m / 4) * kappa_parallel

# Print the final value
print(f"kappa_parallel_100KeV_SOQLT = {kappa_parallel_100KeV} cm^2/s")

# Plot sigma_z^2 / tau^2 vs tau
sigma_z_values = [sigma_z(t) / t**2 if t != 0 else 0 for t in tqdm(tau, desc="Computing sigma_z^2 / tau^2")]

plt.figure(figsize=(6, 5))
plt.plot(tau, sigma_z_values)
plt.xlabel(r'$\tau$', fontsize=14)
plt.ylabel(r'$\widetilde{\sigma _{z}}^{2} / \tau^{2}$', fontsize=14)
plt.title(r'SOQLT $\widetilde{\sigma _{z}}^{2} / \tau^{2}$ vs $\tau$ at 0.1AU')
plt.show()

# Plot the results for D_mu_mu vs mu
fig, ax = plt.subplots(figsize=(6, 5))
plt.legend()
plt.plot(mu_values, D_values, color='k', label='SOQLT')
ax.tick_params(axis='y', which='major', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)
plt.xlabel('μ', fontsize=14)
plt.ylabel(r'$\tilde{D}_{\mu \mu }^{(2)}$', fontsize=14)
plt.title(r'SOQLT of $\tilde{D}_{\mu \mu }^{(2)}$ vs μ at 0.1AU')
plt.show()
